In [1]:
import time
import requests
import datetime
import pandas as pd

from util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'
path_data = './data'
path_csv = path_data + '/csv'
file_name_out_r = path_csv + '/covid19-' + level + '.csv'
file_name_pop_r = path_csv + '/popolazione-regioni.csv'
file_name_icu_r = path_csv + '/terapie-intensive.csv'

In [2]:
# Define today, yesterday and day before yesterday dates
date_tday = datetime.date.today()
date_yday = Functions.get_previous_date(date_tday)
date_bf_yday = Functions.get_previous_date(date_yday)
date_tday, date_yday, date_bf_yday

(datetime.date(2020, 10, 29),
 datetime.date(2020, 10, 28),
 datetime.date(2020, 10, 27))

In [3]:
# Today url for regions file
url_tday = Functions.get_url_date(level, date_tday)
# Yesterday url for regions file
url_yday = Functions.get_url_date(level, date_yday)
# Day before yesterday url for regions file
url_bf_yday = Functions.get_url_date(level, date_bf_yday)

url_tday, url_yday, url_bf_yday

('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201029.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201028.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201027.csv')

In [4]:
if False:
    while True:
        status_code = requests.get(url_tday).status_code
        if status_code == 200:
            print('Presenti dati odierni')
            break
        print('Status code: {}. Retrying...'.format(status_code))
        time.sleep(20)

In [5]:
# Get csv files
try:
    # today and yesterday if today present
    df_raw_r_1 = pd.read_csv(url_tday)
    print('Dati odierni')
    df_raw_r_0 = pd.read_csv(url_yday)
    date_last = date_tday
except Exception as e:
    # yesterday and day before yesterday otherwise
    df_raw_r_1 = pd.read_csv(url_yday)
    df_raw_r_0 = pd.read_csv(url_bf_yday)
    date_last = date_yday
    print('Dati di ieri')

Dati odierni


In [6]:
# State S
df_raw_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-29T17:00:00,ITA,1,Piemonte,45.073274,7.680687,2381,146,2527,25364,...,2104,2585,32807,4332,31834,33196,65030,997507,609925,NaN
1,2020-10-29T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,120,7,127,1457,...,154,178,1235,162,2635,346,2981,40095,25235,NaN
2,2020-10-29T17:00:00,ITA,3,Lombardia,45.466794,9.190347,3355,345,3700,64184,...,6478,7339,92567,17414,143887,33978,177865,2850943,1780305,NaN
3,2020-10-29T17:00:00,ITA,5,Veneto,45.434905,12.338452,759,97,856,22770,...,2026,2109,25247,2371,24522,26722,51244,2288598,890507,NaN
4,2020-10-29T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,152,34,186,3922,...,376,468,5115,387,8609,1001,9610,526798,225536,NaN
5,2020-10-29T17:00:00,ITA,7,Liguria,44.411493,8.932699,907,50,957,6979,...,458,1018,16863,1742,18879,7662,26541,431053,224168,NaN
6,2020-10-29T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1054,123,1177,18536,...,1483,1545,27724,4601,39481,12557,52038,1553171,835830,In seguito a verifica sui dati comunicati nei ...
7,2020-10-29T17:00:00,ITA,9,Toscana,43.769231,11.255889,956,137,1093,22877,...,1610,1966,13678,1310,31190,7768,38958,1056864,706557,NaN
8,2020-10-29T17:00:00,ITA,10,Umbria,43.106758,12.388247,260,41,301,5270,...,572,694,3277,119,2917,6050,8967,290964,169837,NaN
9,2020-10-29T17:00:00,ITA,11,Marche,43.616760,13.518875,247,39,286,4850,...,610,686,6950,1009,13015,80,13095,314006,183944,NaN


In [7]:
# State S - 1
df_raw_r_0.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-28T17:00:00,ITA,1,Piemonte,45.073274,7.680687,2237,135,2372,23415,...,2547,2827,32353,4305,30854,31591,62445,984720,599224,NaN
1,2020-10-28T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,107,7,114,1316,...,63,77,1216,157,2496,307,2803,39314,24935,NaN
2,2020-10-28T17:00:00,ITA,3,Lombardia,45.466794,9.190347,3072,292,3364,58042,...,6522,7558,91763,17357,138923,31603,170526,2808259,1755084,NaN
3,2020-10-28T17:00:00,ITA,5,Veneto,45.434905,12.338452,708,90,798,20802,...,2083,2143,25180,2355,24377,24758,49135,2275869,886165,NaN
4,2020-10-28T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,146,35,181,3551,...,287,406,5028,382,8208,934,9142,521139,223790,NaN
5,2020-10-28T17:00:00,ITA,7,Liguria,44.411493,8.932699,878,46,924,6554,...,521,926,16312,1733,18076,7447,25523,424986,221286,NaN
6,2020-10-28T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,999,119,1118,17112,...,1150,1212,27671,4593,38677,11817,50494,1531311,827354,In seguito a verifica sui dati comunicati nei ...
7,2020-10-28T17:00:00,ITA,9,Toscana,43.769231,11.255889,857,130,987,21373,...,1320,1708,13335,1297,29543,7449,36992,1041270,696653,NaN
8,2020-10-28T17:00:00,ITA,10,Umbria,43.106758,12.388247,255,37,292,4707,...,238,401,3160,114,2723,5550,8273,286380,167083,NaN
9,2020-10-28T17:00:00,ITA,11,Marche,43.616760,13.518875,250,32,282,4244,...,301,351,6875,1008,12409,0,12409,310091,181572,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-29T17:00:00,ITA,1,Piemonte,45.073274,7.680687,2381,146,2527,25364,...,2104,2585,32807,4332,31834,33196,65030,997507,609925,NaN
1,2020-10-29T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,120,7,127,1457,...,154,178,1235,162,2635,346,2981,40095,25235,NaN
2,2020-10-29T17:00:00,ITA,3,Lombardia,45.466794,9.190347,3355,345,3700,64184,...,6478,7339,92567,17414,143887,33978,177865,2850943,1780305,NaN
3,2020-10-29T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,302,24,326,5512,...,150,471,9432,741,12327,3684,16011,510947,230495,NaN
4,2020-10-29T17:00:00,ITA,5,Veneto,45.434905,12.338452,759,97,856,22770,...,2026,2109,25247,2371,24522,26722,51244,2288598,890507,NaN
5,2020-10-29T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,152,34,186,3922,...,376,468,5115,387,8609,1001,9610,526798,225536,NaN
6,2020-10-29T17:00:00,ITA,7,Liguria,44.411493,8.932699,907,50,957,6979,...,458,1018,16863,1742,18879,7662,26541,431053,224168,NaN
7,2020-10-29T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1054,123,1177,18536,...,1483,1545,27724,4601,39481,12557,52038,1553171,835830,In seguito a verifica sui dati comunicati nei ...
8,2020-10-29T17:00:00,ITA,9,Toscana,43.769231,11.255889,956,137,1093,22877,...,1610,1966,13678,1310,31190,7768,38958,1056864,706557,NaN
9,2020-10-29T17:00:00,ITA,10,Umbria,43.106758,12.388247,260,41,301,5270,...,572,694,3277,119,2917,6050,8967,290964,169837,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,11,2020-10-28T17:00:00,ITA,1,Piemonte,45.073274,7.680687,2237,135,2372,...,2547,2827,32353,4305,30854,31591,62445,984720,599224,NaN
1,17,2020-10-28T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,107,7,114,...,63,77,1216,157,2496,307,2803,39314,24935,NaN
2,8,2020-10-28T17:00:00,ITA,3,Lombardia,45.466794,9.190347,3072,292,3364,...,6522,7558,91763,17357,138923,31603,170526,2808259,1755084,NaN
3,19,2020-10-28T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,275,21,296,...,302,381,9116,736,11931,3609,15540,505278,228843,NaN
4,18,2020-10-28T17:00:00,ITA,5,Veneto,45.434905,12.338452,708,90,798,...,2083,2143,25180,2355,24377,24758,49135,2275869,886165,NaN
5,5,2020-10-28T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,146,35,181,...,287,406,5028,382,8208,934,9142,521139,223790,NaN
6,7,2020-10-28T17:00:00,ITA,7,Liguria,44.411493,8.932699,878,46,924,...,521,926,16312,1733,18076,7447,25523,424986,221286,NaN
7,4,2020-10-28T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,999,119,1118,...,1150,1212,27671,4593,38677,11817,50494,1531311,827354,In seguito a verifica sui dati comunicati nei ...
8,15,2020-10-28T17:00:00,ITA,9,Toscana,43.769231,11.255889,857,130,987,...,1320,1708,13335,1297,29543,7449,36992,1041270,696653,NaN
9,16,2020-10-28T17:00:00,ITA,10,Umbria,43.106758,12.388247,255,37,292,...,238,401,3160,114,2723,5550,8273,286380,167083,NaN


In [10]:
# Print raw columns
# print(df_r_1.columns, df_r_0.columns)
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rf_1 = df_r_1.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0 = df_r_0.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
# df_rf_1.columns, df_rf_0.columns

In [11]:
df_rf_0 = df_rf_0.sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0;

In [12]:
df_r = df_rf_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(file_name_pop_r)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-10-29T17:00:00,1,Piemonte,2381,146,2527,25364,27891,2104,2585,32807,4332,31834,33196,65030,997507,609925,4341375
1,2020-10-29T17:00:00,2,Valle d'Aosta,120,7,127,1457,1584,154,178,1235,162,2635,346,2981,40095,25235,125501
2,2020-10-29T17:00:00,3,Lombardia,3355,345,3700,64184,67884,6478,7339,92567,17414,143887,33978,177865,2850943,1780305,10103969
3,2020-10-29T17:00:00,4,Trentino-Alto Adige,302,24,326,5512,5838,150,471,9432,741,12327,3684,16011,510947,230495,1074819
4,2020-10-29T17:00:00,5,Veneto,759,97,856,22770,23626,2026,2109,25247,2371,24522,26722,51244,2288598,890507,4907704
5,2020-10-29T17:00:00,6,Friuli Venezia Giulia,152,34,186,3922,4108,376,468,5115,387,8609,1001,9610,526798,225536,1211357
6,2020-10-29T17:00:00,7,Liguria,907,50,957,6979,7936,458,1018,16863,1742,18879,7662,26541,431053,224168,1543127
7,2020-10-29T17:00:00,8,Emilia-Romagna,1054,123,1177,18536,19713,1483,1545,27724,4601,39481,12557,52038,1553171,835830,4467118
8,2020-10-29T17:00:00,9,Toscana,956,137,1093,22877,23970,1610,1966,13678,1310,31190,7768,38958,1056864,706557,3722729
9,2020-10-29T17:00:00,10,Umbria,260,41,301,5270,5571,572,694,3277,119,2917,6050,8967,290964,169837,880285


In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(file_name_icu_r)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-10-29T17:00:00,1,Piemonte,2381,146,2527,25364,27891,2104,2585,32807,4332,31834,33196,65030,997507,609925,4341375,367
1,2020-10-29T17:00:00,2,Valle d'Aosta,120,7,127,1457,1584,154,178,1235,162,2635,346,2981,40095,25235,125501,20
2,2020-10-29T17:00:00,3,Lombardia,3355,345,3700,64184,67884,6478,7339,92567,17414,143887,33978,177865,2850943,1780305,10103969,983
3,2020-10-29T17:00:00,4,Trentino-Alto Adige,302,24,326,5512,5838,150,471,9432,741,12327,3684,16011,510947,230495,1074819,106
4,2020-10-29T17:00:00,5,Veneto,759,97,856,22770,23626,2026,2109,25247,2371,24522,26722,51244,2288598,890507,4907704,825
5,2020-10-29T17:00:00,6,Friuli Venezia Giulia,152,34,186,3922,4108,376,468,5115,387,8609,1001,9610,526798,225536,1211357,175
6,2020-10-29T17:00:00,7,Liguria,907,50,957,6979,7936,458,1018,16863,1742,18879,7662,26541,431053,224168,1543127,209
7,2020-10-29T17:00:00,8,Emilia-Romagna,1054,123,1177,18536,19713,1483,1545,27724,4601,39481,12557,52038,1553171,835830,4467118,516
8,2020-10-29T17:00:00,9,Toscana,956,137,1093,22877,23970,1610,1966,13678,1310,31190,7768,38958,1056864,706557,3722729,415
9,2020-10-29T17:00:00,10,Umbria,260,41,301,5270,5571,572,694,3277,119,2917,6050,8967,290964,169837,880285,70


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_rf_1['ricoverati_con_sintomi'] - df_rf_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_rf_1['terapia_intensiva'] - df_rf_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_rf_1['totale_ospedalizzati'] - df_rf_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_rf_1['isolamento_domiciliare'] - df_rf_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_rf_1['nuovi_positivi'] - df_rf_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_rf_1['tamponi'] - df_rf_0['tamponi']
df_r['nuovi_casi_testati'] = df_rf_1['casi_testati'] - df_rf_0['casi_testati']
df_r['nuovi_deceduti'] = df_rf_1['deceduti'] - df_rf_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_rf_1['dimessi_guariti'] - df_rf_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-10-29T17:00:00,3,Lombardia,3355,345,3700,64184,67884,6478,7339,...,35.10,6142,-219,42684,25221,57,804,17.19,0.07263,1.76
1,2020-10-29T17:00:00,15,Campania,1297,164,1461,36243,37704,2818,3103,...,38.41,2752,676,17735,10463,20,265,17.50,0.05363,0.84
2,2020-10-29T17:00:00,1,Piemonte,2381,146,2527,25364,27891,2104,2585,...,39.78,1949,-242,12787,10701,27,454,20.22,0.05954,1.50
3,2020-10-29T17:00:00,5,Veneto,759,97,856,22770,23626,2026,2109,...,11.76,1968,-34,12729,4342,16,67,16.57,0.04297,1.04
4,2020-10-29T17:00:00,12,Lazio,1786,168,1954,27758,29712,1766,1995,...,22.49,1647,32,25164,20288,15,214,7.93,0.03401,0.71
5,2020-10-29T17:00:00,9,Toscana,956,137,1093,22877,23970,1610,1966,...,33.01,1504,258,15594,9904,13,343,12.61,0.05281,1.05
6,2020-10-29T17:00:00,8,Emilia-Romagna,1054,123,1177,18536,19713,1483,1545,...,23.84,1424,333,21860,8476,8,53,7.07,0.03459,1.16
7,2020-10-29T17:00:00,7,Liguria,907,50,957,6979,7936,458,1018,...,23.92,425,92,6067,2882,9,551,16.78,0.06597,1.72
8,2020-10-29T17:00:00,19,Sicilia,839,115,954,11791,12745,557,789,...,21.38,501,81,7226,4472,13,219,10.92,0.01588,0.40
9,2020-10-29T17:00:00,16,Puglia,669,78,747,9255,10002,565,716,...,21.31,578,-56,7083,3866,7,144,10.11,0.01786,0.43


In [16]:
# Write pre-processed data
df_r.to_csv(file_name_out_r, index=False)